In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.3/887.3 kB 44.0 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

model = YOLO("yolov8x.pt")

distance_threshold = 100
min_people_in_crowd = 3

video_path = "input_video.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_video_path = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame_count >= 1000:
        break

    frame_count += 1

    results = model(frame)
    people_boxes = []

    for result in results:
        boxes = result.boxes.xyxy
        confidences = result.boxes.conf
        class_ids = result.boxes.cls

        for i in range(len(boxes)):
            x1, y1, x2, y2 = map(int, boxes[i][:4])
            conf = confidences[i]
            class_id = int(class_ids[i])

            if class_id == 0 and conf > 0.1:
                people_boxes.append((x1, y1, x2, y2))
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    crowd_groups = []

    for i, box1 in enumerate(people_boxes):
        group = {i}

        for j, box2 in enumerate(people_boxes):
            if i == j:
                continue

            center1 = ((box1[0] + box1[2]) // 2, (box1[1] + box1[3]) // 2)
            center2 = ((box2[0] + box2[2]) // 2, (box2[1] + box2[3]) // 2)
            distance = np.sqrt((center1[0] - center2[0]) ** 2 + (center1[1] - center2[1]) ** 2)

            if distance < distance_threshold:
                group.add(j)

        if len(group) >= min_people_in_crowd:
            crowd_groups.append(group)

    for group in crowd_groups:
        for idx in group:
            x1, y1, x2, y2 = people_boxes[idx]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    out.write(frame)

cap.release()
out.release()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 131M/131M [00:00<00:00, 334MB/s]



0: 384x640 35 persons, 4 handbags, 1 frisbee, 1 clock, 214.4ms
Speed: 26.0ms preprocess, 214.4ms inference, 738.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 34 persons, 2 backpacks, 2 handbags, 1 frisbee, 1 tennis racket, 53.8ms
Speed: 3.1ms preprocess, 53.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 34 persons, 2 backpacks, 2 handbags, 1 frisbee, 1 tennis racket, 53.7ms
Speed: 3.1ms preprocess, 53.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 34 persons, 2 backpacks, 2 handbags, 2 frisbees, 1 tennis racket, 53.8ms
Speed: 3.2ms preprocess, 53.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 33 persons, 2 handbags, 2 frisbees, 41.6ms
Speed: 2.7ms preprocess, 41.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 31 persons, 2 handbags, 2 frisbees, 41.7ms
Speed: 3.0ms preprocess, 41.7ms inference, 1.5ms postprocess per image at shape (1, 